# Explore Toronto Boroughs and Neighbourhoods

### Install relevant packages

In [1]:
!pip3 install requests
!pip3 install bs4
!pip install requests
!pip3 install geocoder
!pip3 install geopy
!pip3 install matplotlib
!pip3 install sklearn
!pip3 install folium

import requests
from bs4 import BeautifulSoup

### Get Toronto Boroughs/Neighbourhoods

Use historical version of page to avoid having to address any structural changes

In [2]:
# get html
page = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&diff=945633050&oldid=942851379')
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
# read into pandas dataframe
import pandas as pd
df = pd.read_html(str(soup.find_all('table')[1]))[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove unassigned Borough

In [4]:
# Remove unassigned postcodes
df = df[df['Borough']!='Not assigned']
df.reset_index(inplace=True)
df.head()

,index,Postcode,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M6A,North York,Lawrence Heights
4,6,M6A,North York,Lawrence Manor


### Get latitude and longitude for Neighbourhoods

In [5]:
# import geocoder
import geocoder

pc_list = []

for i in range(0,len(df)):
    address='{}, {}, {}, Toronto, Ontario'.format(df.at[i,'Postcode'],df.at[i,'Borough'],df.at[i,'Neighbourhood'])
    g = geocoder.arcgis(address)
    pc_list.append([df.at[i,'Postcode'],df.at[i,'Borough'],df.at[i,'Neighbourhood'], g.json['lat'], g.json['lng']])

In [6]:
# convert list to dataframe
neighbourhoods = pd.DataFrame(data=pc_list,columns=['Postcode','Borough','Neighbourhood','Latitude','Longitude'])
neighbourhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,Lawrence Heights,43.723570,-79.437110
4,M6A,North York,Lawrence Manor,43.722920,-79.431310


In [7]:
# install libs
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [8]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(neighbourhoods['Borough'].unique()),
        neighbourhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 210 neighbourhoods.


In [9]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


In [10]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Enter Foursquare credentials below (remove before upload to repository)

In [39]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


Get latitude and longitude for Downtown Torronto

In [12]:
# select borough and neighbourhood to use "Downtown Torronto"
# M5A	Downtown Toronto	Harbourfront	43.650295	-79.359166
lat = 43.650295
lng = -79.359166

In [13]:
# construct RESTful webservice URL

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=G1GAV4HHFUTMJQFBTV12AXZQOUXZDSA3DNK0SG2QSGZDAJ0N&client_secret=F0OA1XBKYZ5TCUGGVURESVRZIO4W40CD4IJCLTKFWP3MW2YI&v=20180605&ll=43.650295,-79.359166&radius=500&limit=100'

In [14]:
# get venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e75b4d129ce6a001c45b3d7'},
 'response': {'headerLocation': 'The Distillery District',
  'headerFullLocation': 'The Distillery District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 40,
  'suggestedBounds': {'ne': {'lat': 43.6547950045, 'lng': -79.3529584099829},
   'sw': {'lat': 43.6457949955, 'lng': -79.3653735900171}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c05ef964a520bff620e3',
       'name': 'The Distillery Historic District',
       'location': {'address': 'btwn Front, Cherry, Gardiner & Parliament',
        'lat': 43.65024435658077,
        'lng': -79.35932278633118,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65024435658077,
          'lng': -79.35932278633118

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Distillery Historic District,Historic Site,43.650244,-79.359323
1,Arvo,Coffee Shop,43.649963,-79.361442
2,Distillery Sunday Market,Farmers Market,43.650075,-79.361832
3,Cacao 70,Dessert Shop,43.650067,-79.360723
4,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

40 venues were returned by Foursquare.


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
# select neighbourhoods in downtown
downtown_neighbourhoods = neighbourhoods[neighbourhoods.Borough.str.contains('Downtown')]
downtown_neighbourhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
5,M7A,Downtown Toronto,Queen's Park,43.660670,-79.390850
12,M5B,Downtown Toronto,Ryerson,43.657363,-79.378180
13,M5B,Downtown Toronto,Garden District,43.657363,-79.378180
26,M5C,Downtown Toronto,St. James Town,43.670810,-79.373480


In [20]:
# get downtown venues
downtown_venues = getNearbyVenues(names=downtown_neighbourhoods['Neighbourhood'],
                                   latitudes=downtown_neighbourhoods['Latitude'],
                                   longitudes=downtown_neighbourhoods['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


In [21]:
#
print(downtown_venues.shape)
downtown_venues.head()

(2426, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.650295,-79.359166,The Distillery Historic District,43.650244,-79.359323,Historic Site
1,Harbourfront,43.650295,-79.359166,Arvo,43.649963,-79.361442,Coffee Shop
2,Harbourfront,43.650295,-79.359166,Distillery Sunday Market,43.650075,-79.361832,Farmers Market
3,Harbourfront,43.650295,-79.359166,Cacao 70,43.650067,-79.360723,Dessert Shop
4,Harbourfront,43.650295,-79.359166,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop


In [22]:
downtown_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,37,37,37,37,37,37
Bathurst Quay,25,25,25,25,25,25
Berczy Park,62,62,62,62,62,62
CN Tower,67,67,67,67,67,67
Cabbagetown,64,64,64,64,64,64
Central Bay Street,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Christie,11,11,11,11,11,11
Church and Wellesley,85,85,85,85,85,85


In [23]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 226 uniques categories.


In [24]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Adelaide,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.027027,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.054054,0.000000,0.027027,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0000,0.000000,0.000000,0.00,0.027027,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.162162,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.027027,0.000000,0.00,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.027027,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000

In [26]:
# print top 5 venues for each neighbourhood
num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.08
2             Theater  0.05
3              Bakery  0.05
4         Yoga Studio  0.03


----Bathurst Quay----
              venue  freq
0       Coffee Shop  0.20
1              Park  0.08
2              Café  0.08
3  Ramen Restaurant  0.04
4     Boat or Ferry  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1          Restaurant  0.05
2  Seafood Restaurant  0.05
3      Farmers Market  0.03
4              Lounge  0.03


----CN Tower----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.07
2          Restaurant  0.06
3                 Bar  0.04
4                Café  0.04


----Cabbagetown----
         venue  freq
0  Coffee Shop  0.09
1     Pharmacy  0.05
2  Pizza Place  0.05
3    Pet Store  0.03
4        Diner  0.03


----Central Bay Street----
         venue  freq
0  Coffee Shop  0.16
1   Restaurant  0.07
2         Café  0.

In [27]:
def return_most_common_venues(row, num_top_venues):

    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Get top 10 venue types for each neighbourhood

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
cols = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        cols.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        cols.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=cols)
neighbourhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Italian Restaurant,Bakery,Theater,Yoga Studio,Cosmetics Shop,Skating Rink,Sandwich Place,Café,Restaurant
1,Bathurst Quay,Coffee Shop,Park,Café,Harbor / Marina,Diner,Sculpture Garden,Boat or Ferry,Caribbean Restaurant,Sushi Restaurant,Ramen Restaurant
2,Berczy Park,Coffee Shop,Seafood Restaurant,Restaurant,Breakfast Spot,Cocktail Bar,Beer Bar,Farmers Market,Hotel,Bakery,Café
3,CN Tower,Coffee Shop,Italian Restaurant,Restaurant,Café,Bar,French Restaurant,Gym / Fitness Center,Sandwich Place,Electronics Store,Park
4,Cabbagetown,Coffee Shop,Pizza Place,Pharmacy,Pet Store,Italian Restaurant,Café,Restaurant,Pub,Diner,Japanese Restaurant


### Cluster neighbourhoods based on venue types

In [29]:
# set number of clusters
kclusters = 7

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 3, 3, 3, 3, 0, 5, 3, 3], dtype=int32)

In [30]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_neighbourhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,3,Coffee Shop,Bakery,Boat or Ferry,Theater,Farmers Market,Mexican Restaurant,Pub,Shoe Store,Chocolate Shop,Spa
5,M7A,Downtown Toronto,Queen's Park,43.660670,-79.390850,4,Coffee Shop,Sandwich Place,Café,Park,Italian Restaurant,Juice Bar,Restaurant,College Auditorium,Mediterranean Restaurant,Spa
12,M5B,Downtown Toronto,Ryerson,43.657363,-79.378180,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Electronics Store,Italian Restaurant,Furniture / Home Store,Diner
13,M5B,Downtown Toronto,Garden District,43.657363,-79.378180,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Electronics Store,Italian Restaurant,Furniture / Home Store,Diner
26,M5C,Downtown Toronto,St. James Town,43.670810,-79.373480,3,Coffee Shop,Grocery Store,Pizza Place,Restaurant,Bank,Caribbean Restaurant,Filipino Restaurant,Market,Breakfast Spot,Food & Drink Shop


### Plot clustered neighbourhoods on map

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Explore neighbourhood clusters

In [32]:
# explore cluster 0
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
142,Downtown Toronto,0,Café,Bakery,Restaurant,Coffee Shop,Bar,Bookstore,Gym,Japanese Restaurant,Italian Restaurant,Greek Restaurant
143,Downtown Toronto,0,Café,Gym,Restaurant,Coffee Shop,Bar,Bakery,Pub,Bookstore,Italian Restaurant,Japanese Restaurant
151,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Bar,Café,Pizza Place,Asian Restaurant,Yoga Studio,Sushi Restaurant,French Restaurant,Gym
152,Downtown Toronto,0,Café,Bar,Vietnamese Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Bakery,Mexican Restaurant,Dumpling Restaurant,Noodle House,Bubble Tea Shop
153,Downtown Toronto,0,Bar,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Bakery,Chinese Restaurant,Dumpling Restaurant,Burger Joint


Cafes and coffee shops, restaurants and bars indicate a dining district for cluster 0

In [33]:
# explore cluster 1
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,Downtown Toronto,1,Pier,Harbor / Marina,Park,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
61,Downtown Toronto,1,Pier,Harbor / Marina,Park,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
62,Downtown Toronto,1,Pier,Harbor / Marina,Park,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


This area would best be described as the Marina (cluster 1)

In [34]:
# explore cluster 2
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
182,Downtown Toronto,2,Playground,Grocery Store,Park,Candy Store,Doner Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


Cluster 2 is surrounded by playgrounds/parks, eastern/african influenced restaurants

In [35]:
# explore cluster 3
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Coffee Shop,Bakery,Boat or Ferry,Theater,Farmers Market,Mexican Restaurant,Pub,Shoe Store,Chocolate Shop,Spa
12,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Electronics Store,Italian Restaurant,Furniture / Home Store,Diner
13,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Electronics Store,Italian Restaurant,Furniture / Home Store,Diner
26,Downtown Toronto,3,Coffee Shop,Grocery Store,Pizza Place,Restaurant,Bank,Caribbean Restaurant,Filipino Restaurant,Market,Breakfast Spot,Food & Drink Shop
36,Downtown Toronto,3,Coffee Shop,Seafood Restaurant,Restaurant,Breakfast Spot,Cocktail Bar,Beer Bar,Farmers Market,Hotel,Bakery,Café
40,Downtown Toronto,3,Coffee Shop,Restaurant,Café,Hotel,Bakery,Italian Restaurant,Gym,Japanese Restaurant,Breakfast Spot,Pub
48,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Bakery,Theater,Yoga Studio,Cosmetics Shop,Skating Rink,Sandwich Place,Café,Restaurant
49,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Asian Restaurant,Gym,Hotel,Japanese Restaurant,Breakfast Spot,Steakhouse,Gastropub
50,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Asian Restaurant,Gym,Hotel,Japanese Restaurant,Breakfast Spot,Steakhouse,Gastropub
73,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,Bar,Gastropub,American Restaurant,Italian Restaurant,Japanese Restaurant


Cluster 3 is defined by coffee shops, clothing and cosmetics shops, Italian Restaurants, cafe's and bars. It looks like a retail and restaurant area. 

In [36]:
# Cluster 4
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,4,Coffee Shop,Sandwich Place,Café,Park,Italian Restaurant,Juice Bar,Restaurant,College Auditorium,Mediterranean Restaurant,Spa
164,Downtown Toronto,4,Coffee Shop,Park,Café,Harbor / Marina,Diner,Sculpture Garden,Boat or Ferry,Caribbean Restaurant,Sushi Restaurant,Ramen Restaurant


These neighbourhoods are located near parks (Cluster 4).

In [37]:
# Cluster 5
downtown_merged.loc[downtown_merged['Cluster Labels'] == 5, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,Downtown Toronto,5,Café,Grocery Store,Baby Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Candy Store,Playground,Empanada Restaurant,Donut Shop


This neighbourhood is on the outskirts of downtown, has close grocery and baby stores. Looks like an inner city family oriented neighbourhood (Cluster 5).

In [38]:
# Cluster 6
downtown_merged.loc[downtown_merged['Cluster Labels'] == 6, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
165,Downtown Toronto,6,Coffee Shop,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Tunnel,Park,Boat or Ferry


This neighbourhood is near the airport (Cluster 6)